In [1]:
import torch, detectron2

In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import mlflow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [3]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("vehicles_train", {}, "Vehicle detection.v9i.coco/train/_annotations.coco.json", "Vehicle detection.v9i.coco/train")
register_coco_instances("v", {}, "Vehicle detection.v9i.coco/test/_annotations.coco.json", "Vehicle detection.v9i.coco/test")
register_coco_instances("vehicles_val", {}, "Vehicle detection.v9i.coco/valid/_annotations.coco.json", "Vehicle detection.v9i.coco/valid")

In [4]:
from detectron2.config import get_cfg, CfgNode

cfg = get_cfg()

cfg.MLFLOW = CfgNode()
cfg.MLFLOW.EXPERIMENT_NAME = "Vehicle Object Detection"
cfg.MLFLOW.RUN_DESCRIPTION = "training with 6000 iterations, 20k images"
cfg.MLFLOW.RUN_NAME = "#4 training" # TODO: Исправить на автосмену
cfg.MLFLOW.TRACKING_URI = "http://localhost:5000"

In [5]:
from detectron2.engine import HookBase
import mlflow

class MLflowHook(HookBase):
    """
    A custom hook class that logs artifacts, metrics, and parameters to MLflow.
    """

    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg.clone()

    def before_train(self):
        with torch.no_grad():
            mlflow.set_tracking_uri(self.cfg.MLFLOW.TRACKING_URI)
            mlflow.set_experiment(self.cfg.MLFLOW.EXPERIMENT_NAME)
            mlflow.start_run(run_name=self.cfg.MLFLOW.RUN_NAME)
            mlflow.set_tag("mlflow.note.content",
                           self.cfg.MLFLOW.RUN_DESCRIPTION)
            for k, v in self.cfg.items():
                mlflow.log_param(k, v)

    def after_step(self):
        with torch.no_grad():
            latest_metrics = self.trainer.storage.latest()
            for k, v in latest_metrics.items():
                mlflow.log_metric(key=k, value=v[0], step=v[1])

    def after_train(self):
        with torch.no_grad():
            with open(os.path.join(self.cfg.OUTPUT_DIR, "model-config.yaml"), "w") as f:
                f.write(self.cfg.dump())
            mlflow.log_artifacts(self.cfg.OUTPUT_DIR)

In [6]:
from detectron2.engine import DefaultTrainer

class CocoTrainer(DefaultTrainer):
    """
    A custom trainer class that evaluates the model on the validation set every `_C.TEST.EVAL_PERIOD` iterations.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs(cfg.OUTPUT_DIR_VALIDATION_SET_EVALUATION,
                        exist_ok=True)
        
        return COCOEvaluator(dataset_name, distributed=False, output_dir=cfg.OUTPUT_DIR_VALIDATION_SET_EVALUATION)

In [7]:
from datetime import datetime
import datetime
import os

from detectron2.engine import DefaultTrainer

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("vehicles_train",)
cfg.DATASETS.TEST= ("vehicles_val",)
cfg.OUTPUT_DIR = ("4_output") # TODO: Исправить на автосмену
cfg.OUTPUT_DIR_VALIDATION_SET_EVALUATION = os.path.join(
        cfg.OUTPUT_DIR, "validation-set-evaluation")
cfg.OUTPUT_DIR_TEST_SET_EVALUATION = os.path.join(
        cfg.OUTPUT_DIR, "test-set-evaluation")
cfg.TEST.EVAL_PERIOD = 100
cfg.DATALOADER.NUM_WORKERS = 12
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 20  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 6000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
os.makedirs(cfg.OUTPUT_DIR_VALIDATION_SET_EVALUATION, exist_ok=True)
os.makedirs(cfg.OUTPUT_DIR_TEST_SET_EVALUATION, exist_ok=True)

#setup_logger(output=os.path.join(cfg.OUTPUT_DIR, "training-log.txt"))

mlflow_hook = MLflowHook(cfg)

trainer = CocoTrainer(cfg)
trainer.register_hooks(hooks=[mlflow_hook])
trainer.resume_or_load(resume=False)
trainer.train()

[03/06 14:02:46 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (10, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (10,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (36, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (36,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

[03/06 14:02:47 d2.engine.train_loop]: Starting training from iteration 0


/home/forssh/workspace/.venv/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/06 14:03:14 d2.utils.events]:  eta: 1:57:06  iter: 19  total_loss: 2.637  loss_cls: 2.071  loss_box_reg: 0.09347  loss_rpn_cls: 0.3656  loss_rpn_loc: 0.09955    time: 1.1751  last_time: 1.1845  data_time: 0.0771  last_data_time: 0.0477   lr: 4.9953e-06  max_mem: 13695M


2024-03-06 14:03:15.269369: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 14:03:15.313267: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 14:03:15.313306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 14:03:15.315059: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 14:03:15.323921: I tensorflow/core/platform/cpu_feature_guar

[03/06 14:03:42 d2.utils.events]:  eta: 1:57:13  iter: 39  total_loss: 2.471  loss_cls: 1.913  loss_box_reg: 0.1048  loss_rpn_cls: 0.3396  loss_rpn_loc: 0.08848    time: 1.1858  last_time: 1.1790  data_time: 0.0614  last_data_time: 0.0683   lr: 9.9902e-06  max_mem: 13695M
[03/06 14:04:08 d2.utils.events]:  eta: 1:56:28  iter: 59  total_loss: 2.008  loss_cls: 1.567  loss_box_reg: 0.09429  loss_rpn_cls: 0.2486  loss_rpn_loc: 0.08162    time: 1.1849  last_time: 1.1341  data_time: 0.0579  last_data_time: 0.0432   lr: 1.4985e-05  max_mem: 13695M
[03/06 14:04:35 d2.utils.events]:  eta: 1:56:47  iter: 79  total_loss: 1.719  loss_cls: 1.076  loss_box_reg: 0.1065  loss_rpn_cls: 0.3406  loss_rpn_loc: 0.08687    time: 1.1912  last_time: 1.2317  data_time: 0.0572  last_data_time: 0.0799   lr: 1.998e-05  max_mem: 13695M
WARNING [03/06 14:05:00 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/06 14:05:00 d2.data.datasets.coco]: Loa

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

#setup_logger(output=os.path.join(cfg.OUTPUT_DIR_TEST_SET_EVALUATION, "evaluation-log.txt"))

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

predictor = DefaultPredictor(cfg)

evaluator = COCOEvaluator("v", output_dir=cfg.OUTPUT_DIR_TEST_SET_EVALUATION)
test_set_loader = build_detection_test_loader(cfg, "v")

evaluation_results = inference_on_dataset(predictor.model, test_set_loader, evaluator)
#logging.info("Evaluation results on test set: %s", evaluation_results)

for k, v in evaluation_results["bbox"].items():
    mlflow.log_metric(f"Test Set {k}", v, step=0)

mlflow.log_artifacts(cfg.OUTPUT_DIR_TEST_SET_EVALUATION, "test-set-evaluation")
mlflow.log_text(str(evaluation_results), "test-set-evaluation/coco-metrics.txt")

In [ ]:
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import Visualizer
from PIL import Image
import os
import numpy as np

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65   # set a custom testing threshold
predictor = DefaultPredictor(cfg)


dataset_name = 'vehicles_test'
dataset_metadata = MetadataCatalog.get(dataset_name)


input_images = [
    "Vehilce detection.v3i.coco/train/1ff7910c-16884491480894_jpeg.rf.70dbfcc5722372608719d20b68a71541.jpg",
    "Vehilce detection.v3i.coco/train/fe27f76c-electricScooter_114_jpg.rf.6dfaaaa049ee65ef79cb656a14d3a35f.jpg",
    "Vehilce detection.v3i.coco/train/f7ab2b5f-52_jpg.rf.93d3972b5242280d38f273f74a001319.jpg",
    "Vehilce detection.v3i.coco/train/d951a446-i_jpg.rf.e4682497eda25d2d639e83d1d655d4fc.jpg",
    "Vehilce detection.v3i.coco/train/b7fcd547-60d072afa40cf3d4e89a928913630124_jpeg.rf.a49d2a003e30c8b32865f3d43f4933bb.jpg",
    "Vehilce detection.v3i.coco/train/846cf55e-42_jpg.rf.ad0ad373bdc619d1e1771b549c72f4cb.jpg",
    "Vehilce detection.v3i.coco/train/4c9f0301-monowheel_12_jpg.rf.dbdfdfa6a003e1aea4ab9e69aa208def.jpg",
    "Vehilce detection.v3i.coco/train/4e42ee0c-_-_-_10_jpg.rf.d8cb4a8f3ecd5146efd79270376b954d.jpg",
    "Vehilce detection.v3i.coco/train/ant_sales-1030_png_jpg.rf.e6122844adc05593c86d492d2b818543.jpg",
    "Vehilce detection.v3i.coco/valid/image25_png_jpg.rf.8ad9ec59347c435970f2142ff7074cbe.jpg",
    "Vehilce detection.v3i.coco/valid/screenshot_17440_jpg.rf.0fa15167b99fcc4380da5e0433a63ed9.jpg",
    "Vehilce detection.v3i.coco/valid/screenshot_13131_jpg.rf.603f483cb49a9f7e300979b26f65e6e3.jpg",
    "Vehilce detection.v3i.coco/valid/screenshot_1341_jpg.rf.0e6e1343b17c6f262ac96e211636c427.jpg",
    "Vehilce detection.v3i.coco/valid/image07_png_jpg.rf.4f472e89f7ba03ac51d8ed81c8c9e897.jpg",
]


output_dir = cfg.OUTPUT_DIR_TEST_SET_EVALUATION
os.makedirs(output_dir, exist_ok=True)


for input_image_path in input_images:
    im = Image.open(input_image_path)
    im = np.array(im)
    outputs = predictor(im)

    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    vis_output = v.draw_instance_predictions(outputs["instances"].to("cpu"))


    output_image = vis_output.get_image()[:, :, ::-1]


    output_image_path = os.path.join(output_dir, os.path.basename(input_image_path))
    
 
    Image.fromarray(output_image).save(output_image_path)




In [ ]:
mlflow.end_run()

In [ ]:
! jupyter notebook stop